# Snapchat Political Ads
* **See the main project notebook for instructions to be sure you satisfy the rubric!**
* See Project 03 for information on the dataset.
* A few example prediction questions to pursue are listed below. However, don't limit yourself to them!
    * Predict the reach (number of views) of an ad.
    * Predict how much was spent on an ad.
    * Predict the target group of an ad. (For example, predict the target gender.)
    * Predict the (type of) organization/advertiser behind an ad.

Be careful to justify what information you would know at the "time of prediction" and train your model using only those features.

# Summary of Findings


### Introduction
TODO

### Baseline Model
TODO

### Final Model
TODO

### Fairness Evaluation
TODO

# Code

In [20]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

In [139]:
eighteen = pd.read_csv("2018.csv")
nineteen = pd.read_csv("2019.csv")
result = pd.concat([eighteen, nineteen])
result = result.reset_index(drop=True)

Prediction: We will try to predict the reach, or the number of views, of an ad. This is a regression problem. 

### Baseline Model

In [186]:
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from datetime import datetime
import re
from sklearn import metrics

In [141]:
# changing all dates to datetime objects
result['StartDate'] = result['StartDate'].str[:-1]
result['EndDate'] = result['EndDate'].str[:-1]
result['StartDate'] = pd.to_datetime(result['StartDate'])
result['EndDate'] = pd.to_datetime(result['EndDate'], errors='coerce')


In [142]:
# EndDate                         fill with "Present" or current time
result['EndDate'] = result['EndDate'].fillna(datetime.utcnow())
# Gender                          fill with "All"
result['Gender'] = result['Gender'].fillna("All")
# display(result[['StartDate', 'EndDate']])
result['Duration'] = abs(result['EndDate']- result['StartDate'])
result ["Duration"] = round(result['Duration'].apply(lambda x: x.seconds / 60))

result["MediaType"] = result['CreativeUrl'].str.extract('.+mediaType=([A-z0-9]+)', expand=True)
result["MediaType"] = result["MediaType"].str.lower()
result["MediaType"] = result["MediaType"].str.replace("jpeg", "jpg")
# result["MediaType"].value_counts()
result = result.drop("CreativeUrl",axis=1)
result
# Quantitative: Impressions, Spend, Duration
# Nominal: CountryCode, Gender, MediaType

,ADID,Spend,Impressions,StartDate,EndDate,OrganizationName,BillingAddress,CandidateBallotInformation,PayingAdvertiserName,Gender,...,Segments,LocationType,Language,AdvancedDemographics,Targeting Connection Type,Targeting Carrier (ISP),Targeting Geo - Postal Code,CreativeProperties,Duration,MediaType
0,58be4f8f92574108faaa4c014b579a9de2afb06410add9...,2048,186004,2018-10-18 19:42:18,2018-11-06 23:00:00.000000,Bully Pulpit Interactive,"1140 Connecticut Ave NW, Suite 800,Washington,...",NaN,NextGen America,All,...,NaN,NaN,NaN,NaN,NaN,NaN,"89120,89169,89119,89108,89110,89103,89160,8916...",NaN,198.0,mp4
1,82288288e3e363f5e61cb23bacec3831888132c17193c2...,1025,335092,2018-08-25 14:00:00,2018-09-08 13:59:59.000000,"Tubemogul, Inc.","1250 53rd Street,,,Emeryville,94608,US",NaN,Liberal Party,FEMALE,...,Provided by Advertiser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1440.0,mp4
2,bc873a18a6bc1d6b17df5f4d699a621b7a84d14a26f499...,924,399010,2018-10-09 15:51:44,2018-11-07 03:00:00.000000,Penna Powers,"1706 s major street,salt lake city,84115,US",NaN,Lieutenant Governor's Office,All,...,Provided by Advertiser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,668.0,mp4
3,8d6bd945e484a50d899364c618d8e30b8918813a2877c8...,1096,365035,2018-08-25 14:00:00,2018-09-08 13:59:59.000000,"Tubemogul, Inc.","1250 53rd Street,,,Emeryville,94608,US",NaN,Liberal Party,FEMALE,...,Provided by Advertiser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1440.0,mp4
4,14535fea019a9b1a910a77ce1555af8bdedbb5c78fb60a...,6743,3149886,2018-10-06 01:11:41,2018-11-07 03:00:00.000000,Lockwood Strategy,US,NaN,Change Now,All,...,Provided by Advertiser,NaN,NaN,NaN,NaN,NaN,NaN,web_view_url:https://action.socalhealthcarecoa...,108.0,png
5,e28a51b84d15841990c3476cfaa2d6d943a6f207c6d1b9...,4486,1211144,2018-09-06 13:42:15,2018-10-17 06:30:00.000000,Bully Pulpit Interactive,"1140 Connecticut Ave NW, Suite 800,Washington,...",NaN,NextGen America,All,...,NaN,NaN,NaN,NaN,NaN,NaN,"32307,33620,32816,32306,33965,32304,32603,3260...",web_view_url:https://nextgenamerica.org/on-the...,1008.0,mp4
6,d937d4d5a83c8988a1550cf86d2b94e3913eb791de0d00...,852,138288,2018-10-17 15:00:00,2018-11-07 04:00:00.000000,Democratic Congressional Campaign Committee,"430 S Capitol St SE,Washington,20003,US",NaN,DCCC,All,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,web_view_url:https://mypollingplace.org/,780.0,mp4
7,74d3118b9410ec06629609008cd63097df9ed58d78c171...,1000,187796,2018-10-23 19:02:42,2018-11-21 07:59:59.000000,The Modesto Bee,"948 11th Street, Suite 300,Modesto,95354,US",NaN,STOPtheWaterGrabCoalition,All,...,Provided by Advertiser,NaN,NaN,"Occupation (Farming, Fish or Forestry),HHI: Hi...",NaN,NaN,NaN,web_view_url:http://bit.ly/2MMlKfn,777.0,mp4
8,93b515dee410cd2470de04c272743ee5adcb6a4456e3e0...,49,17329,2018-11-02 22:47:04,2018-11-07 01:00:00.000000,Mothership Strategies,"1328 Florida Avenue NW, Building C, Washington...",NaN,Voto Latino,All,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,web_view_url:https://vota2018.org/,133.0,png
9,ea2a199530b600fe1cc159053ea260c86ed71f132903cd...,1343,1429115,2018-11-05 09:43:16,2018-11-30 09:33:55.000000,Republic of Media Ltd.,"4th Floor, Nova House,Edinburgh,EH3 9QQ,GB",NaN,Scottish Government,All,...,Provided by Advertiser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1431.0,mp4


In [143]:
result = result[['Impressions', 'Duration', 'Spend','Gender', 'CountryCode', 'MediaType']]

In [144]:
X = result.drop(['Impressions'], axis=1)
y = result.Impressions

In [176]:
# TODO
types = X.dtypes
catcols = types.loc[types == np.object].index
numcols = types.loc[types != np.object].index

cats = Pipeline([
    ('imp', SimpleImputer(strategy='constant', fill_value='NULL')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False)),
])

ct = ColumnTransformer([
    ('catcols', cats, catcols),
    ('numcols', SimpleImputer(strategy='constant', fill_value=0), numcols)
])

pl = Pipeline([('feats', ct), ('reg', LinearRegression())])

# X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.25)
# pl.fit(X_tr, y_tr)
# rsquared = pl.score(X_ts, y_ts)
# print("r-squared: " + str(rsquared))
# preds = pl.predict(X_ts)
# rmse = np.sqrt(np.mean(preds - y_ts)**2)
# print("rmse: " + str(rmse))
out = []
for _ in range(100):
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.25)
    pl.fit(X_tr, y_tr)
    rsquared = pl.score(X_ts, y_ts)
    out.append(rsquared)
np.mean(out)

0.5855230551982886

### Final Model

In [178]:
from sklearn.ensemble import RandomForestRegressor

cats = Pipeline([
    ('imp', SimpleImputer(strategy='constant', fill_value='NULL')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False)),
    ('pca', PCA(svd_solver='full', n_components=0.99))
])

ct = ColumnTransformer([
    ('catcols', cats, catcols),
    ('numcols', SimpleImputer(strategy='constant', fill_value=0), numcols)
])

pl2 = Pipeline([('feats', ct), ('reg', RandomForestRegressor())])

In [171]:
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.25)
pl2.fit(X_tr, y_tr)
rsquared = pl2.score(X_ts, y_ts)
print("r-squared: " + str(rsquared))
preds = pl2.predict(X_ts)
rmse = np.sqrt(np.mean(preds - y_ts)**2)
print("rmse: " + str(rmse))

r-squared: 0.7761983178246616
rmse: 1962.3144653916409


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [172]:
out = []
for _ in range(100):
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.25)
    pl.fit(X_tr, y_tr)
    rsquared = pl.score(X_ts, y_ts)
    out.append(rsquared)
out

0.5275037377042573
0.6918812718190275
0.4975249929764637
0.7091463391695924
0.7742680068061658
0.44454471204628576
0.6513295019870153
0.5965867049940712
0.6763952303562757
0.5254621068379153
0.6777716682493626
0.7463478335942668
0.7862682912320709
0.4295726748458355
0.7354124021020231
0.7037114851538051
0.6786403632455645
0.807542716313653
0.5678331705780404
0.584591606355036
0.851945945929701
0.6509457676867239
0.7431394754708998
0.6327170558456896
0.39235935864395555
0.439602916015527
0.5370405397162068
0.5308295466769246
0.7327565209706771
0.7529017542234223
0.6793893835771807
0.40205794377271975
0.514525117181567
0.8067950584625896
0.760001886870033
0.7427949591520557
0.5823427944302815
0.7547357399217918
0.6978301583832127
0.8266256436797351
0.6834166526657766
0.8431592383694642
0.7892985981989261
0.7358552551260953
0.5885085430182498
0.70812497368031
0.5429982799455282
0.70367285907678
0.681389833114558
0.8106243418726085
0.6979595439855957
0.7978043126366801
0.7261226098882821
0

[0.5275037377042573,
 0.6918812718190275,
 0.4975249929764637,
 0.7091463391695924,
 0.7742680068061658,
 0.44454471204628576,
 0.6513295019870153,
 0.5965867049940712,
 0.6763952303562757,
 0.5254621068379153,
 0.6777716682493626,
 0.7463478335942668,
 0.7862682912320709,
 0.4295726748458355,
 0.7354124021020231,
 0.7037114851538051,
 0.6786403632455645,
 0.807542716313653,
 0.5678331705780404,
 0.584591606355036,
 0.851945945929701,
 0.6509457676867239,
 0.7431394754708998,
 0.6327170558456896,
 0.39235935864395555,
 0.439602916015527,
 0.5370405397162068,
 0.5308295466769246,
 0.7327565209706771,
 0.7529017542234223,
 0.6793893835771807,
 0.40205794377271975,
 0.514525117181567,
 0.8067950584625896,
 0.760001886870033,
 0.7427949591520557,
 0.5823427944302815,
 0.7547357399217918,
 0.6978301583832127,
 0.8266256436797351,
 0.6834166526657766,
 0.8431592383694642,
 0.7892985981989261,
 0.7358552551260953,
 0.5885085430182498,
 0.70812497368031,
 0.5429982799455282,
 0.70367285907678,

### Fairness Evaluation

In [187]:
# X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.25)
# pl2.fit(X_tr, y_tr)
# pl2.score(X_ts,y_ts)
out = []
for _ in range(100):
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.25)
    pl2.fit(X_tr, y_tr)
    rsquared = pl2.score(X_ts, y_ts)
    out.append(rsquared)
np.mean(out)

0.6603818861789408

### Permutation Testing

In [208]:
temp = X_ts
# temp['age_bracket'] = temp['Impressions'].apply(lambda x:5*(x//5 + 1))
temp['prediction'] = preds
temp['tag'] = y_ts

(
    temp
    .groupby('ImpressionsBracket')
    .apply(lambda x:1 - metrics.recall_score(x.tag, x.prediction))
    .plot(kind='bar')
)
temp

KeyError: 'age'